# Assistant Skill

In [124]:
import requests
import os
import re
import json
import openai
from dotenv import load_dotenv


## Retrieve environment variables
load_dotenv()

openai.api_key = OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

MEALIE_API_KEY = os.getenv("MEALIE_API_KEY")
MEALIE_URL = os.getenv("MEALIE_URL")
MEALIE_LIST_ID = os.getenv("MEALIE_LIST_ID")

NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_DATABASE_ID_TASKS = os.getenv("NOTION_DATABASE_ID_TASKS")
NOTION_TASK_ID_GENERAL = os.getenv("NOTION_TASK_ID_GENERAL")

## Headers for authentication
OPENAI_HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}
OPENROUTER_HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}
MEALIE_HEADERS = {
    "Authorization": f"Bearer {MEALIE_API_KEY}",
    "Content-Type": "application/json"
}
NOTION_HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

## API settings for selected model
# MODEL = "gpt-4o-mini"
# MODEL = "nvidia/llama-3.1-nemotron-ultra-253b-v1:free"
MODEL = "google/gemini-2.0-flash-thinking-exp:free"

if MODEL == "gpt-4o-mini":
    MODEL_API_URL = "https://api.openrouter.ai/v1/generate"
    MODEL_HEADERS = OPENAI_HEADERS
else:
    MODEL_API_URL = "https://openrouter.ai/api/v1/chat/completions"
    MODEL_HEADERS = OPENROUTER_HEADERS

### Add Item to Mealie List

In [125]:
## Get all Mealie shopping lists
response = requests.get(f"{MEALIE_URL}/households/shopping/lists", headers=MEALIE_HEADERS)
lists = response.json()["items"]
[{"name": list["name"], "id": list["id"]} for list in lists]

[{'name': 'List', 'id': '56fb8d06-18b6-4d8a-abba-6afec00930d6'}]

In [126]:
payload = {
  "shoppingListId": MEALIE_LIST_ID,
  "note": "tomatoes"
}

response = requests.post(f"{MEALIE_URL}/households/shopping/items", headers=MEALIE_HEADERS, data=json.dumps(payload))

if response.status_code == 201:
    print(f"Successfully added: {payload['note']}")
else:
    print(f"Failed to add {payload['note']}: {response.status_code}, {response.text}")

Successfully added: tomatoes


In [127]:
def add_to_mealie_list(item):
    try:
        ## Create the payload for the API request
        payload = {
            "shoppingListId": MEALIE_LIST_ID,
            "note": item
        }

        ## Send the request to add the item to the shopping list
        response = requests.post(f"{MEALIE_URL}/households/shopping/items", 
                                 headers=MEALIE_HEADERS, data=json.dumps(payload))

        if response.status_code == 201:
            return f"Successfully added {item}"
        else:
            return f"Unsuccessful adding {item}: {response.status_code}, {response.text}"
        
    except Exception as e:
        return f"Error occurred: {str(e)}"

In [128]:
query = "add 1 cup of sugar"
item = re.sub(r"^add\s+", "", query, flags=re.I).strip()

add_to_mealie_list(item)

'Successfully added 1 cup of sugar'

### Add Item to Notion Database

In [129]:
def create_notion_task(title, parent_task_id=NOTION_TASK_ID_GENERAL):
    try:
        # Create the payload for the API request
        payload = {
            "parent": {"database_id": NOTION_DATABASE_ID_TASKS},
            "properties": {
                "Name": {
                    "title": [
                        {
                            "text": {
                                "content": title
                            }
                        }
                    ]
                },
                "Parent task": {
                    "relation": [
                        {
                            "id": parent_task_id
                        }
                    ]
                }
            }
        }

        # Send the request to create the page
        response = requests.post("https://api.notion.com/v1/pages", 
                                 headers=NOTION_HEADERS, 
                                 data=json.dumps(payload))

        if response.status_code == 200:
            print(f"Page '{title}' created successfully.")
        else:
            print(f"Failed to create page '{title}': {response.status_code}, {response.text}")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

In [131]:
create_notion_task("This is a test")

Page 'This is a test' created successfully.


### Smart add to Mealie or Notion

In [133]:
def smart_add_item(input_value):
    try:
        ## Define the prompt
        prompt = f"""
        You are a smart assistant. Analyze the following input and split it into individual items or tasks. 
        For each item or task, determine if it is a shopping item or a task. 
        Respond with a JSON array where each element is a dictionary with "function" and "item" attributes.
        The output format should be a raw JSON array without formatting (such as code block) that can be interpretable with json.loads().
        "function" should be "add_to_mealie_list" for shopping items and "create_notion_task" for tasks.
        Input: "{input_value}"
        """

        ## Call model to analyze the input
        payload = {
            "model": MODEL,
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(MODEL_API_URL, 
                                 headers=MODEL_HEADERS, data=json.dumps(payload))

        if response.status_code != 200:
            return f"Error calling {payload['model']}: {response.status_code}, {response.text}"

        ## Parse the response
        gpt_response = response.json()
        gpt_content = gpt_response["choices"][0]["message"]["content"]
        decisions = json.loads(gpt_content)
        print(decisions)

        mealie_items = []
        notion_items = []

        ## Execute the appropriate function for each decision
        for decision in decisions:
            if decision["function"] == "add_to_mealie_list":
                add_to_mealie_list(decision["item"])
                mealie_items.append(decision["item"])
            elif decision["function"] == "create_notion_task":
                create_notion_task(decision["item"])
                notion_items.append(decision["item"])
            print(f"Decision: {decision['function']} - {decision['item']}")

        ## Construct the result string
        mealie_str = f"added {', '.join(mealie_items)} to mealie" if mealie_items else ""
        notion_str = f"added {', '.join(notion_items)} to notion" if notion_items else ""
        result_str = " and ".join(filter(None, [mealie_str, notion_str]))

        return result_str

    except Exception as e:
        return f"Error occurred: {str(e)}"

In [134]:
query = "add 3 potatoes and mow the lawn, and I need to file taxes and pick up oranges and lemons"
input_value = re.sub(r"^add\s+", "", query, flags=re.I).strip()
        
smart_add_item(input_value)

[{'function': 'add_to_mealie_list', 'item': '3 potatoes'}, {'function': 'create_notion_task', 'item': 'mow the lawn'}, {'function': 'create_notion_task', 'item': 'file taxes'}, {'function': 'add_to_mealie_list', 'item': 'oranges'}, {'function': 'add_to_mealie_list', 'item': 'lemons'}]
Decision: add_to_mealie_list - 3 potatoes
Page 'mow the lawn' created successfully.
Decision: create_notion_task - mow the lawn
Page 'file taxes' created successfully.
Decision: create_notion_task - file taxes
Decision: add_to_mealie_list - oranges
Decision: add_to_mealie_list - lemons


'added 3 potatoes, oranges, lemons to mealie and added mow the lawn, file taxes to notion'

### Read shopping list

In [135]:
response = requests.get(f"{MEALIE_URL}/households/shopping/items?orderBy=checked&orderDirection=asc&page=1&perPage=100&checked=false", 
                        headers=MEALIE_HEADERS)

print(response.text)

{"page":1,"per_page":100,"total":26,"total_pages":1,"items":[{"quantity":0.0,"unit":null,"food":{"id":"a2d19a59-e857-4c46-baf5-7e7ad3840d06","name":"sesame oil","pluralName":null,"description":"","extras":{},"labelId":"ddab9d19-45dd-4ad2-8c84-d61fe2a14f6f","aliases":[],"householdsWithIngredientFood":["huang-household"],"label":{"name":"Seasoning","color":"#8B4513","groupId":"97cd1a62-a4f0-4de5-815d-3882fef2d948","id":"ddab9d19-45dd-4ad2-8c84-d61fe2a14f6f"},"createdAt":"2025-03-26T05:32:28.996417Z","updatedAt":"2025-03-26T05:32:28.996419Z"},"note":"99 Ranch","isFood":true,"disableAmount":false,"display":"sesame oil 99 Ranch","shoppingListId":"56fb8d06-18b6-4d8a-abba-6afec00930d6","checked":false,"position":8,"foodId":"a2d19a59-e857-4c46-baf5-7e7ad3840d06","labelId":"ddab9d19-45dd-4ad2-8c84-d61fe2a14f6f","unitId":null,"extras":{},"id":"31b49ba2-e57b-4b89-b3e4-2cb7bd676eb6","groupId":"97cd1a62-a4f0-4de5-815d-3882fef2d948","householdId":"7ad19b33-562d-40fd-9d92-4b350bebfb6d","label":{"name